# **Tutorial 9: Paleoclimate Data Assimilation**
**Week 1, Day 4, Paleoclimate**

**Content creators:** Sloane Garelick

**Content reviewers:** Brodie Pearson

**Content editors:** Agustina Pesce

**Production editors:** TBD

**Our 2023 Sponsors:** TBD

### **Code and Data Sources**

The code for this notebook is based on [code available from Erb et al. (2022)](https://github.com/Holocene-Reconstruction/Holocene-code) and workflow presented during the [Paleoclimate Data Assimilation Workshop 2022](https://github.com/michaelerb/da-workshop).

Data from the following sources are used in this tutorial:

*   Matthew B. Osman, Jessica E. Tierney, Jiang Zhu, Robert Tardif, Gregory J. Hakim, Jonathan King, Christopher J. Poulsen. 2021. Globally resolved surface temperatures since the Last Glacial Maximum. Nature, 599, 239-244. http://doi.org/10.1038/s41586-021-03984-4.
*   King, J. M., Tierney, J., Osman, M., Judd, E. J., & Anchukaitis, K. J. (2023). DASH: A MATLAB Toolbox for Paleoclimate Data Assimilation. Geoscientific Model Development, preprint. https://doi.org/10.5194/egusphere-2023-68.



















# **Tutorial 9 Objectives**

As we discussed in the video, proxies and models both have advantages and limitations for reconstructing past changes in earth's climate system. One approach for combining the strengths of both paleoclimate proxies and models is data assimilation. In this tutorial, we'll look at paleoclimate reconstructions that were made with data assimilation. The specific reconstruction we'll be analyzing is the Last Glacial Maximum reanalysis (LGMR) from [Osman et al. (2021)](https://www.nature.com/articles/s41586-021-03984-4), which contains temperature and δ<sup>18</sup>O data for the past 24,000 years.


During this tutorial you will:

*   Plot a time series of a paleoclimate data assimilation reconstruction
*   Create global maps and zonal mean plots of temperature anomalies 
*   Assess how and why LGM to present temperature anomalies vary with latitude






In [ ]:
# # Install libraries
# !pip install --no-binary shapely shapely --force #this allows us to load cartopy
# !pip install cartopy
# !pip install pooch
# !pip install xarray

In [ ]:
# Import libraries
import pooch
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

import cartopy.crs as ccrs
import cartopy.util as cutil

### Load the LGMR paleoclimate data assimilation reconstruction

This dataset contains reconstructions of surface air temperature (SAT), δ<sup>18</sup>O, and global mean surface temperature (GMST). This data spans the past 24,000 years. The period of time from ~24,000 to 19,000 years ago is referred to as the Last Glacial Maximum (LGM). The LGM was the most recent glacial period in Earth's history. During this time, northern hemisphere ice sheets were larger, global sea level was lower, atmospheric CO<sub>2</sub> was lower and global temperature were cooler. 


We will calculate the LGM to present global mean temperature from a paleoclimate data assimilation to asses how Earth's climate varied over the past 24,000 uyear.

First let's download the paleoclimate data assimilation reconstruction for surface air temperature (SAT). 

In [ ]:
data_path= pooch.retrieve(
  url="https://www.ncei.noaa.gov/pub/data/paleo/reconstructions/osman2021/LGMR_SAT_climo.nc",
  known_hash=None,
)

dataset = xr.open_dataset(data_path)

### Plotting a **time series** of the paleoclimate data assimilation

Now that the data is loaded, we can plot a time series of the temperature data to beginning to assess global changes. However, the dimensions of the `sat_mean` variable are age-lat-lon, so we first need to calculate a global mean.

In [ ]:
# Calculate the global mean surface temperature
tas_global_mean = dataset.sat.mean(dim=['lat', 'lon'])
tas_global_mean

Now that we calculated our global mean, we can plot the results as a time series to assess global changes in temperature over the past 24,000 years:

In [ ]:
# Plot the global mean surface temperature since the LGM
f,ax1 = plt.subplots(1, 1, figsize=(12,6))
ax1.plot(dataset['age'], tas_global_mean, linewidth=3)

ax1.set_xlim(dataset['age'].max().values, dataset['age'].min().values)
ax1.set_ylabel('$\Delta$T ($^\circ$C)', fontsize=16)
ax1.set_xlabel('Age (yr BP)', fontsize=16)
ax1.set_title(
  f'Mean $\Delta$T ($^\circ$C) for LGMR, region: {region}',
  fontsize=18,
  loc='center'
)
plt.show()

Consider the following questions:


*   How has global temperature varied over the past 24,000 years?
*   What climate forcings may have contributed to the increase in temperature ~17,000 years ago? 



### Plotting a **temperature anomaly map** of the paleoclimate data assimilation

Data assimilation creates spatial reconstructions, so we can also make figures showing spatial temperature anomalies for different time periods (i.e., the change in temperature between two specified times). The anomaly that we'll interpret is the difference between global temperature from 20,5000 to 21,500 years ago (i.e. "LGM") and 0 to 1,000 years ago (i.e. "modern") . First, we'll calculate the average temperatures for each time period.

In [ ]:
#Calculate the LGM (21,500-20,500 year) mean temperature
lgm = dataset.sel(
    age=slice('20500', '21500'),lon=slice(0, 357.5), lat=slice(-90, 90)
)
lgm_mean = lgm.mean(dim='age')

#Calculate the "modern" (0-1000 year) mean temperature
modern = dataset.sel(
    age=slice('0', '1000'),lon=slice(0, 357.5), lat=slice(-90, 90)
)
modern_mean = modern.mean(dim='age')

Now we can calculate the anomaly and create a map to visualized the change in temperature from the LGM to present in different parts on Earth.

In [ ]:
sat_change = lgm_mean-modern_mean

In [ ]:
# Make a map of changes
plt.figure(figsize=(12,8))
ax = plt.axes(projection=ccrs.Robinson())
ax.set_global()
sat_change.sat.plot(
    ax=ax,
    transform=ccrs.PlateCarree(), x="lon", y="lat", 
    cbar_kwargs={'orientation': 'horizontal', 'label':'$\Delta$T ($^\circ$C)'}
)
ax.coastlines()
ax.set_title(
    f'$\Delta$T ($^\circ$C) for LGMR, ages: anom = {ages_anom}, ref = {ages_ref}',
    loc='center',
    fontsize=16
)
ax.gridlines(color='k',linewidth=1,linestyle=(0,(1,5)))
ax.spines['geo'].set_edgecolor('black')
plt.show()

Before we interpret this data, another useful way to visualize this data is through a plot of zonal mean temperature (the average temperature for all locations at a single latitude). Once we calculate this zonal mean, we can create a plot of LGM to present temperature anomalies versus latitude.

In [ ]:
zonal_mean = sat_change.sat.mean(dim='lon')
latitude = dataset.lat

In [ ]:
# Make a zonal mean figure of the changes
fig, ax1 = plt.subplots(1, 1)
plt.plot(zonal_mean,latitude)
ax1.axvline(x=0,color='gray',alpha=1,linestyle=':',linewidth=2)
ax1.set_ylim(-90, 90)
ax1.set_xlabel('$\Delta$T ($^\circ$C)')
ax1.set_ylabel('Latitude ($^\circ$)')
ax1.set_title(
    f'Zonal-mean $\Delta$T ($^\circ$C), ages: anom = {ages_anom}, ref = {ages_ref}',
    loc='center',
    )
plt.show()

Looking at both the map and zonal mean plot, consider the following questions: 

*   How does the temperature anomaly vary with latitude?
*   What forcings and processes may cause the spatial patterns in temperature change since the LGM?
*   Where was the largest temperature change? Why might this region region have undergone the largest temperature change during this time?